# Variables

## Copy on write

In [1]:
library(pryr)

Registered S3 method overwritten by 'pryr':
  method      from
  print.bytes Rcpp



In [2]:
x <- 158
address(x)

[1] "0x5598bc45e520"

In [3]:
y <- x
address(y)

[1] "0x5598bc45e520"

In [4]:
identical(address(x), address(y))

[1] TRUE

In [5]:
x <- 86
address(x)
address(y)

[1] "0x5598baec7518"

[1] "0x5598bc45e520"

In [6]:
identical(address(x), address(y))

[1] FALSE

## Préallocation

In [7]:
lcg <- function(a, b, m,
                seed = 0)
  function(){
    ret <- (a * seed + b) %% m
    seed <<- ret
    ret / m
  }

In [8]:
## LCG de numerical recipes.
nr_lcg <- lcg(1664525, 1013904223, 2^32)

rand_prealloc <- function(n){
    res <- numeric(n)
    for (kk in 1:n)
        res[kk] <- nr_lcg()

    res
}

rand_vec <- function(n){
    replicate(n, nr_lcg())
}

In [9]:
library(microbenchmark)
print(microbenchmark(rand_vec(1e4), rand_prealloc(1e4)))

Unit: milliseconds
                 expr       min        lq     mean   median       uq      max
      rand_vec(10000) 14.475049 15.914283 19.36851 18.25837 20.57277 39.75269
 rand_prealloc(10000)  9.132864  9.768021 11.89094 11.32383 12.93549 36.17871
 neval
   100
   100


# Fonctions

## De niveau supérieur

In [10]:
Filter(function(x) identical(x %% 2, 0), 1:10)

[1]  2  4  6  8 10

## Composition

In [11]:
formals(lcg)

$a


$b


$m


$seed
[1] 0


In [12]:
body(lcg)

function() {
    ret <- (a * seed + b)%%m
    seed <<- ret
    ret/m
}

In [13]:
environment(lcg)

<environment: R_GlobalEnv>

## Portée lexicale

In [14]:
x <- 42
f <- function(){
    print(x)
    x <- 666
    print(x)
}
f()

[1] 42
[1] 666


In [15]:
print(x)

[1] 42


## Variable globale vs. closure

In [16]:
seed <- 42
lcg_bad <- function(a, b, m)
  function(){
    ret <- (a * seed + b) %% m
    seed <<- ret
    ret / m
  }

In [17]:
nr_lcg <- lcg(1664525, 1013904223, 2^32)

In [18]:
identical(environment(lcg), globalenv())

[1] TRUE

In [19]:
identical(environment(nr_lcg), globalenv())

[1] FALSE

# Comparaisons

## Approche impérative

In [20]:
add1 <- function(x) x + 1

add2 <- function(x) x + 2

for_add1_twice <- function(n){
    res <- integer(n)
    for (kk in 1:n)
        res[n] <- add1(add1(rpois(1, 1)))

    res
}

for_add2 <- function(n){
    res <- integer(n)
    for (kk in 1:n)
        res[n] <- add2(rpois(1, 1))

    res
}

for_noCall <- function(n){
    res <- integer(n)
    for (kk in 1:n)
        res[n] <- rpois(1, 1) + 2

    res
}

In [21]:
print(microbenchmark(for_add1_twice(1e4),
                     for_add2(1e4),
                     for_noCall(1e4)))

Unit: milliseconds
                  expr      min       lq     mean   median       uq      max
 for_add1_twice(10000) 22.59052 24.35937 29.85741 28.41422 33.37950 52.05460
       for_add2(10000) 18.62884 21.99016 24.60918 23.49123 26.42475 38.86438
     for_noCall(10000) 15.33360 16.42019 20.97512 19.53979 23.33472 77.72734
 neval
   100
   100
   100


## Approche fonctionnelle

In [22]:
sapply_add1_twice <- function(n)
    sapply(rpois(n, 1), function(x) add1(add1(x)))

sapply_add2 <- function(n) sapply(rpois(n, 1), add2)

sapply_noCall <- function(n) sapply(rpois(n, 1), `+`, y = 1)

In [23]:
print(microbenchmark(sapply_add1_twice(1e4),
                     sapply_add2(1e4),
                     sapply_noCall(1e4)))

Unit: milliseconds
                     expr       min        lq      mean    median        uq
 sapply_add1_twice(10000) 12.610106 14.322282 17.523976 16.422704 18.845928
       sapply_add2(10000)  5.923592  6.537617  8.334495  7.607203  9.740279
     sapply_noCall(10000)  3.562126  4.054324  5.576478  4.715336  6.704204
      max neval
 74.82576   100
 19.28138   100
 16.68560   100


## Approche array

In [24]:
add2_correct <- function(n) rpois(n, 1) + 2

In [25]:
print(microbenchmark(for_add1_twice(1e4),
                     for_add2(1e4),
                     for_noCall(1e4),
                     sapply_add1_twice(1e4),
                     sapply_add2(1e4),
                     sapply_noCall(1e4),
                     add2_correct(1e4)))

Unit: microseconds
                     expr       min         lq      mean     median        uq
    for_add1_twice(10000) 22492.384 24004.8500 26791.855 25280.8655 28505.266
          for_add2(10000) 18623.574 19694.3270 23506.022 22939.7485 24516.531
        for_noCall(10000) 15322.088 15887.9310 19400.966 18013.0120 20362.406
 sapply_add1_twice(10000) 12356.952 13161.7660 14448.113 13457.9685 15331.476
       sapply_add2(10000)  5803.650  6109.7400  6803.235  6361.7850  6827.063
     sapply_noCall(10000)  3562.638  3783.6515  4431.018  3955.2390  4387.776
      add2_correct(10000)   372.209   379.0275   406.939   383.8115   388.418
       max neval
 37660.913   100
 51790.916   100
 63619.062   100
 21242.988   100
 14584.365   100
 13182.177   100
  2104.472   100
